In [9]:
import pickle
import glob

path = 'data/*.bin'

files = glob.glob(path)

signals = []

for f in files:
    with open(f, 'rb') as fp:
        signal = pickle.load(fp)
        signals.append(signal)

In [10]:
#pad the 2d data to make it symmetric
from keras.preprocessing.sequence import pad_sequences

signals_2 = []
for signal in signals:
    signal_2 = pad_sequences(signal, dtype='float32') 
    signals_2.append(signal_2)

In [11]:
import pandas as pd

signals = []
for signal in signals_2:
    data_0 = pd.DataFrame(signal)
    data_0.columns = data_0.columns.astype(str)
    signals.append(data_0)

In [12]:
for signal in signals[1:]:
    for x in range(signal.shape[1], 477):
        signal[str(x)] = float(0)

In [13]:
label = 1
for signal in signals:
    signal['label'] = label
    label = label + 1

In [14]:
big_data = pd.concat(signals)

big_data

,0,1,2,3,4,5,6,7,8,9,...,468,469,470,471,472,473,474,475,476,label
0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,60.351273,58.827641,57.969322,58.617294,58.412926,54.821445,53.179749,51.250446,50.785027,1
1,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,2.224843,1.474851,0.970638,0.391956,1
2,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.382016,0.983828,1.599483,2.945693,1
3,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,64.867752,62.752323,61.224167,60.169281,59.643188,57.596199,57.373947,57.028549,56.849007,1
4,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,28.877832,28.950697,29.149450,30.874146,31.006813,30.445852,29.826515,28.119081,28.211523,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,3.03394,2.313159,2.315145,2.218729,3.014631,2.645271,2.418086,2.030883,2.078114,2.141873,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5
115,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5
116,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5
117,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5


In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.framework import ops
from sklearn.preprocessing import OneHotEncoder
import tensorflow.compat.v1 as tf1
tf1.disable_v2_behavior() 
import tensorflow as tf2 

df = big_data

df.label = df.label.shift(-1)

# parameters
time_steps = 1
inputs = 477
outputs = 5

# remove nans as a result of the shifted values
df = df.iloc[:-1,:]

# convert to numoy
df = df.values

# X_y_split
train_X = df[:, 1:]
train_y = df[:, 0]

# center and scale
scaler = MinMaxScaler(feature_range=(0, 1))    
train_X = scaler.fit_transform(train_X)

# reshape input to 3D array
train_X = train_X[:,None,:]

onehot_encoder = OneHotEncoder()
encode_categorical = train_y.reshape(len(train_y), 1)
train_y = onehot_encoder.fit_transform(encode_categorical).toarray()

learning_rate = 0.001
epochs = 500
batch_size = int(train_X.shape[0]/2)
length = train_X.shape[0]
display = 100
neurons = 100

# clear graph (if any) before running
ops.reset_default_graph()

X = tf2.compat.v1.placeholder(tf2.float32, [None, time_steps, inputs])
y = tf2.compat.v1.placeholder(tf2.float32, [None, outputs])

# LSTM Cell
cell = tf2.compat.v1.nn.rnn_cell.BasicLSTMCell(num_units=neurons, activation=tf2.compat.v1.nn.relu)
cell_outputs, states = tf2.compat.v1.nn.dynamic_rnn(cell, X, dtype=tf2.compat.v1.float32)

# pass into Dense layer
stacked_outputs = tf2.reshape(cell_outputs, [-1, neurons])
out = tf2.compat.v1.layers.dense(inputs=stacked_outputs, units=outputs)

# squared error loss or cost function for linear regression
loss = tf2.compat.v1.reduce_mean(tf2.compat.v1.nn.softmax_cross_entropy_with_logits_v2(
        labels=y, logits=out))
# optimizer to minimize cost
optimizer = tf2.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

accuracy = tf2.compat.v1.metrics.accuracy(labels =  tf2.compat.v1.argmax(y, 1),
                          predictions = tf2.compat.v1.argmax(out, 1),
                          name = "accuracy")
precision = tf2.compat.v1.metrics.precision(labels=tf2.compat.v1.argmax(y, 1),
                                 predictions=tf2.compat.v1.argmax(out, 1),
                                 name="precision")
recall = tf2.compat.v1.metrics.recall(labels=tf2.compat.v1.argmax(y, 1),
                           predictions=tf2.compat.v1.argmax(out, 1),
                           name="recall")
f1 = 2 * accuracy[1] * recall[1] / ( precision[1] + recall[1] )

with tf2.compat.v1.Session() as sess:
    # initialize all variables
    tf2.compat.v1.global_variables_initializer().run()
    tf2.compat.v1.local_variables_initializer().run()
    # Train the model
    for steps in range(epochs):
        mini_batch = zip(range(0, length, batch_size),
                   range(batch_size, length+1, batch_size))

        # train data in mini-batches
        for (start, end) in mini_batch:
            sess.run(training_op, feed_dict = {X: train_X[start:end,:,:],
                                               y: train_y[start:end,:]})
        # print training performance 
        if (steps+1) % display == 0:
            # evaluate loss function on training set
            loss_fn = loss.eval(feed_dict = {X: train_X, y: train_y})
            print('Step: {}  \tTraining loss (mse): {}'.format((steps+1), loss_fn))

    # Test model
    acc, prec, recall, f1 = sess.run([accuracy, precision, recall, f1],
                                     feed_dict = {X: train_X, y: train_y})

    print('Accuracy:', acc[1])
    print('Precision:', prec[1])
    print('Recall:', recall[1])
    print('F1 score:', f1)

Step: 100  	Training loss (mse): 0.009098670445382595
Step: 200  	Training loss (mse): 0.002118688076734543
Step: 300  	Training loss (mse): 0.0007342171156778932
Step: 400  	Training loss (mse): 0.00022690916375722736
Step: 500  	Training loss (mse): 9.537798905512318e-05
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 score: 1.0
